In [ ]:
from graph_tool.all import *
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from Temporal_Community_Detection import temporal_network
import random
import pickle
import os
import matplotlib

In [ ]:
comms1 = 6
comms2 = 3

layers = 2

fixed_size = int(abs(np.random.normal(30,10)))

fixed_rate1 = int(abs(np.random.normal(20,8)))
fixed_rate2 = int(abs(np.random.normal(20,8)))# spike rate per commiunity

path = '/projects/academic/smuldoon/bengieru/Community_Detection/M_FSFR/DSBM/'

os.makedirs(path, exist_ok = True)

comm_sizes = [[fixed_size for i in range(comms1)], #layer1 community sizes
              [3*fixed_size,fixed_size,2*fixed_size]] #layer2 community sizes
spike_rates = [[fixed_rate1 for i in range(comms1)],
              [fixed_rate2 for i in range(comms2)]]

with open(path + "comm_size.pkl", "wb") as fp:
    pickle.dump(comm_sizes, fp)

window_size = 1000 # size, in frames, each adjacency matrix correspond to. better to be equal to bin_size 
standard_dev = 1.2 # for gaussian kernel
k = 5 #for jittering the spikes
pad = True
num_neurons = int(sum(comm_sizes[0]))

display_truth(comm_sizes, community_operation = 'merge')
plt.savefig(path + 'Ground_truths.pdf')

In [ ]:
spikes = create_time_series('merge', comm_sizes, spike_rates, windowsize = window_size, k = k)
plt.savefig(path + 'spiketrain.pdf')

In [ ]:
with open(path + "spikes.pkl", "wb") as fp:
    pickle.dump(spikes, fp)

In [ ]:
fig,ax = plt.subplots(1, 1, figsize = (10,10))
n, bins = spike_count(spikes, ax)
plt.savefig(path + 'spike_distribution.pdf')

In [ ]:
binned_spikes = bin_time_series(spikes, window_size, gaussian = True, sigma = standard_dev)
fig,ax = plt.subplots(layers, 1, figsize=(20,50))
for i in range(layers):
    ax[i].imshow(binned_spikes[i], origin = 'lower', interpolation='nearest', aspect='auto')
    ax[i].set_title('Gaussian Spikes (Layer %d)'%(i+1), fontsize = 20)
    ax[i].set_xlabel('TIME (in Miliseconds)', fontsize = 20)
    ax[i].set_xticks([j*100 for j in range(11)])
    ax[i].set_yticks([j*10 for j in range(int(num_neurons/10)+1)])
    ax[i].set_ylabel('Neuron ID', fontsize = 25)
    ax[i].set_xlabel('Time (Frames)', fontsize = 20)
    ax[i].tick_params(axis = 'both', labelsize = 20)
plt.tight_layout()
plt.savefig(path + 'binned_spiketrain.pdf')

In [ ]:
##create cross-correlation matrices that are the adjacency matrices of the network at each layer
adjacency_matrices = []
for i in range(layers):
    adjacency_matrices.append(cross_correlation_matrix(binned_spikes[i])[0])
    
if pad:
    padded_adjacencies = [adjacency_matrices[0]]  + adjacency_matrices + [adjacency_matrices[-1]]
    layers = layers + 2

In [ ]:
fig,ax = plt.subplots(2, 2, figsize = (32,32))
for i in range(2):
    for j in range(2):
        k = ax[i][j].imshow(padded_adjacencies[i*2+j], 
                            origin = 'lower', 
                            interpolation='nearest', 
                            aspect='auto',  
                            extent = [0,num_neurons,0,num_neurons])
        ax[i][j].set_title('Adjacency Matrix (Layer %d)'%(i*2+j +1), fontsize = 35)
        ax[i][j].set_xticks([k*10 for k in range(int(num_neurons/10)+1)])
        ax[i][j].set_yticks([k*10 for k in range(int(num_neurons/10)+1)])
        ax[i][j].tick_params(axis = 'both', labelsize = 20)
fig.suptitle('Community merge with sizes %d,%d,%d,%d,%d,%d and spike rates %d,%d,%d,%d,%d,%d for 1st layer and %d,%d,%d for 2nd layer'%(comm_sizes[0][0],comm_sizes[0][1],comm_sizes[0][2],comm_sizes[0][3],comm_sizes[0][4],comm_sizes[0][5],spike_rates[0][0],spike_rates[0][1],spike_rates[0][2],spike_rates[0][3],spike_rates[0][4],spike_rates[0][5],spike_rates[1][0],spike_rates[1][1],spike_rates[1][2]), fontsize = 45)
cbar = fig.colorbar(k, ax = ax.flat, orientation = 'horizontal')
cbar.ax.tick_params(labelsize = 25) 
plt.savefig(path+'adjacency.pdf')

In [ ]:
TN = temporal_network(num_neurons, 
                      layers, 
                      window_size, 
                      data = 'list__adjacency', 
                      list_adjacency = padded_adjacencies, 
                      omega = 1, 
                      kind = 'ordinal')

fig,ax = plt.subplots(1,1, figsize = (25,15))
TN.raster_plot(spikes, ax)
plt.savefig(path + 'raster_plot.pdf')

In [ ]:
grid = 26
threshs = np.linspace(0, 0.5, grid)

path_deg_corr = path + 'deg_corr/'

os.makedirs(path_deg_corr, exist_ok = True)

In [ ]:
processed_matrices = {}
for k, f in enumerate(threshs):
    edge_lists = [[] for i in range(layers)]
    for i in range(layers):
        A = padded_adjacencies[i]
        firing = np.transpose(np.nonzero(A))
        for j,m in enumerate(firing):
            if A[m[0],m[1]]<f: pass
            else: 
                quadreplet =(m[0], m[1], A[m[0], m[1]], i)
                edge_lists[i].append(quadreplet)
    processed_matrices['%.2f'%f] = edge_lists

In [ ]:
all_states = {}
for k,f in enumerate(threshs):
    graphs = []

    g = Graph()
    graphs.append(g)
    graphs[0].add_vertex(num_neurons)
    e_weight = graphs[0].new_ep("double")
    e_layer = graphs[0].new_ep("int")
    n_id = graphs[0].new_vp("int", vals = [i for i in range(num_neurons)])
    graphs[0].add_edge_list(processed_matrices['%.2f'%f][0], eprops=[e_weight, e_layer])
    graphs[0].edge_properties["edge_weight"] = e_weight
    graphs[0].edge_properties["edge_layer"] = e_layer
    
    
    G = graphs[0]

    for l in range(1,layers):
        g = Graph()
        graphs.append(g)
        graphs[l].add_vertex(num_neurons)
        e_weight = graphs[l].new_ep("double")
        e_layer = graphs[l].new_ep("int")
        n_id = graphs[l].new_vp("int", vals = [i for i in range(num_neurons)])
        graphs[l].add_edge_list(processed_matrices['%.2f'%f][l], eprops=[e_weight, e_layer])
        graphs[l].edge_properties["edge_weight"] = e_weight
        graphs[l].edge_properties["edge_layer"] = e_layer
        
        G = graph_union(G, graphs[l], include = False, internal_props = True)
        
    states = []

    state = LayeredBlockState(G, deg_corr = True, ec = G.ep.edge_layer,  recs=[G.ep.edge_weight], rec_types=["real-exponential"],  layers = True, overlap = True)
    
    S1 = state.entropy()
    states.append(state)
    # Equilibrate
    mcmc_equilibrate(state, force_niter=50, mcmc_args=dict(niter=10))

    dls = []         # description length history
    bs = []          # partitions
    S2 = state.entropy()
    def collect_partitions(s):
        global bs, dls
        bs.append(s.get_state())
        dls.append(s.entropy())

    # Now we collect 2000 partitions; but the larger this is, the
    # more accurate will be the calculation
    
    states.append(state)
    
    mcmc_equilibrate(state, force_niter=50, mcmc_args=dict(niter=5), callback=collect_partitions)
    
    S3 = state.entropy()
    states.append(state)
    all_states['%.2f'%f] = states
    print(S1,S2,S3, S2-S1, S3-S2, S3-S1)

In [ ]:
memberships_0 = {}
for k,f in enumerate(threshs):
    node_ids = []
    for i,e in enumerate(all_states['%.2f'%f][0].get_nonoverlap_blocks()):
        node_ids.append(e)
    number_of_colors = len(np.unique(node_ids))

    membership = [[] for i in range(number_of_colors)]
    for i in range(num_neurons):#(num_neurons*layers):
        for j in range(layers):
            node_id = node_ids[j*num_neurons+i]
            membership[node_id].append((i,j))
    memberships_0['%.2f'%f] = membership

memberships_1 = {}
for k,f in enumerate(threshs):
    node_ids = []
    for i,e in enumerate(all_states['%.2f'%f][1].get_nonoverlap_blocks()):
        node_ids.append(e)
    number_of_colors = len(np.unique(node_ids))

    membership = [[] for i in range(number_of_colors)]
    for i in range(num_neurons):#(num_neurons*layers):
        for j in range(layers):
            node_id = node_ids[j*num_neurons+i]
            membership[node_id].append((i,j))
    memberships_1['%.2f'%f] = membership

memberships_2 = {}
for k,f in enumerate(threshs):
    node_ids = []
    for i,e in enumerate(all_states['%.2f'%f][2].get_nonoverlap_blocks()):
        node_ids.append(e)
    number_of_colors = len(np.unique(node_ids))

    membership = [[] for i in range(number_of_colors)]
    for i in range(num_neurons):#(num_neurons*layers):
        for j in range(layers):
            node_id = node_ids[j*num_neurons+i]
            membership[node_id].append((i,j))
    memberships_2['%.2f'%f] = membership

In [ ]:
with open(path_deg_corr + "DSBM_memberships_deg_corr_0.pkl", "wb") as fp:
    pickle.dump(memberships_0, fp)
with open(path_deg_corr + "DSBM_memberships_deg_corr_1.pkl", "wb") as fp:
    pickle.dump(memberships_1, fp)
with open(path_deg_corr + "DSBM_memberships_deg_corr_2.pkl", "wb") as fp:
    pickle.dump(memberships_2, fp)

In [ ]:
fig,ax = plt.subplots(len(threshs),3,figsize = (30,10*len(thresholds)))

for k,f in enumerate(threshs):
    comms_0 = np.zeros((num_neurons,layers))
    color_0 = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(memberships_0['%.2f'%f]))]
    
    comms_1 = np.zeros((num_neurons,layers))
    color_1 = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(memberships_1['%.2f'%f]))]
    
    comms_2 = np.zeros((num_neurons,layers))
    color_2 = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(memberships_2['%.2f'%f]))]
    
    for i, l in enumerate(memberships_0['%.2f'%f]):
        for j,m in enumerate(l):
            comms_0[m[0]][m[1]] = i
            
    for i, l in enumerate(memberships_1['%.2f'%f]):
        for j,m in enumerate(l):
            comms_1[m[0]][m[1]] = i
    
    for i, l in enumerate(memberships_2['%.2f'%f]):
        for j,m in enumerate(l):
            comms_2[m[0]][m[1]] = i
            
    cmap_0 = mpl.colors.ListedColormap(color_0)
    cmap_1 = mpl.colors.ListedColormap(color_1)
    cmap_2 = mpl.colors.ListedColormap(color_2)

    ax[k][0].imshow(comms_0, interpolation = 'none', cmap = cmap_0, aspect = 'auto', origin = 'lower')
    ax[k][1].imshow(comms_1, interpolation = 'none', cmap = cmap_1, aspect = 'auto', origin = 'lower')
    ax[k][2].imshow(comms_2, interpolation = 'none', cmap = cmap_2, aspect = 'auto', origin = 'lower')
    
    ax[k][0].set_xticks([i for i in range(layers)])
    ax[k][0].set_yticks([i*10 for i in range(int(num_neurons/10)+1)])
    ax[k][0].tick_params(axis = 'both', labelsize = 12)
    ax[k][0].set_xlabel('Layers (Time)', fontsize = 25)
    ax[k][0].set_ylabel('Neuron ID', fontsize = 25)
    ax[k][0].set_title('%d Communities, threshold:%.3f'%(len(memberships_0['%.2f'%f]),threshs[k]), fontsize=29)
    
    ax[k][1].set_xticks([i for i in range(layers)])
    ax[k][1].set_yticks([i*10 for i in range(int(num_neurons/10)+1)])
    ax[k][1].tick_params(axis = 'both', labelsize = 12)
    ax[k][1].set_xlabel('Layers (Time)', fontsize = 25)
    ax[k][1].set_ylabel('Neuron ID', fontsize = 25)
    ax[k][1].set_title('%d Communities, threshold:%.3f'%(len(memberships_1['%.2f'%f]),threshs[k]), fontsize=29)
    
    ax[k][2].set_xticks([i for i in range(layers)])
    ax[k][2].set_yticks([i*10 for i in range(int(num_neurons/10)+1)])
    ax[k][2].tick_params(axis = 'both', labelsize = 12)
    ax[k][2].set_xlabel('Layers (Time)', fontsize = 25)
    ax[k][2].set_ylabel('Neuron ID', fontsize = 25)
    ax[k][2].set_title('%d Communities, threshold:%.3f'%(len(memberships_2['%.2f'%f]),threshs[k]), fontsize=29)
plt.tight_layout()
plt.savefig(path_deg_corr + 'communities.pdf')